In [ ]:
import pandas as pd
from unidecode import unidecode

In [ ]:
from google.colab import files

with open('example.txt', 'w') as f:
  f.write('some content')

files.download('example.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
df_movie = pd.read_csv('IMDB-Dataset.csv')
df_movie.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [ ]:
df_movie['sentiment'] = df_movie['sentiment'].map({'positive': 1, 'negative': 0})
df_movie.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [ ]:
#Data cleaning

import re
import html
from bs4 import BeautifulSoup

def lower(review):
    return review.lower()

def deletePunctuation(x):
    return re.sub(r'[^\w\s]', ' ', x)

def deleteSpaces(x):
    return re.sub(r'\s+', ' ', x)

def unescape(x):
    return html.unescape(x)

def deleteHtmlTags(x):
    return BeautifulSoup(x, "lxml").text

def cleanData1(X):
    X.review = X.review.apply(lower)
    X.review = X.review.apply(deleteHtmlTags)
    X.review = X.review.apply(unidecode)
    X.review = X.review.apply(deletePunctuation)
    X.review = X.review.apply(unescape)
    X.review = X.review.apply(deleteSpaces)

print(df_movie)
cleanData1(df_movie)

print(df_movie[:10])

                                                  review  sentiment
0      One of the other reviewers has mentioned that ...          1
1      A wonderful little production. <br /><br />The...          1
2      I thought this was a wonderful way to spend ti...          1
3      Basically there's a family where a little boy ...          0
4      Petter Mattei's "Love in the Time of Money" is...          1
...                                                  ...        ...
49995  I thought this movie did a down right good job...          1
49996  Bad plot, bad dialogue, bad acting, idiotic di...          0
49997  I am a Catholic taught in parochial elementary...          0
49998  I'm going to have to disagree with the previou...          0
49999  No one expects the Star Trek movies to be high...          0

[50000 rows x 2 columns]
                                              review  sentiment
0  one of the other reviewers has mentioned that ...          1
1  a wonderful little producti

In [ ]:
#Data Cleaning 2
import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

stop_words = set(stopwords.words("english")) 
lemmatizer = WordNetLemmatizer()


def clean_text(text):
    text = [lemmatizer.lemmatize(token) for token in text.split(" ")]
    text = [lemmatizer.lemmatize(token, "v") for token in text]
    text = [word for word in text if not word in stop_words]
    text = " ".join(text)
    return text

df_movie['Processed_Reviews'] =df_movie.review.apply(lambda x: clean_text(x))
df_movie.head(5)

LookupError: ignored

In [ ]:
df_movie.Processed_Reviews.apply(lambda x: len(x.split(" "))).mean()

123.17134

In [ ]:
#Train And Save Model

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense , Input , LSTM , Embedding, Dropout , Activation, GRU, Flatten
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model, Sequential
from keras.layers import Convolution1D
from keras import initializers, regularizers, constraints, optimizers, layers

max_features = 6000
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(df_movie['Processed_Reviews'])
list_tokenized_train = tokenizer.texts_to_sequences(df_movie['Processed_Reviews'])

maxlen = 130
X_t = pad_sequences(list_tokenized_train, maxlen=maxlen)
y = df_movie['sentiment']

embed_size = 128
model = Sequential()
model.add(Embedding(max_features, embed_size))
model.add(Bidirectional(LSTM(32, return_sequences = True)))
model.add(GlobalMaxPool1D())
model.add(Dense(20, activation="relu"))
model.add(Dropout(0.05))
model.add(Dense(1, activation="sigmoid"))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

batch_size = 100
epochs = 3
model.fit(X_t,y, batch_size=batch_size, epochs=epochs, validation_split=0.2)
model.save('sentiment-score-model')

Epoch 1/3
400/400 [==============================] - 38s 90ms/step - loss: 0.3629 - accuracy: 0.8366 - val_loss: 0.2724 - val_accuracy: 0.8854
Epoch 2/3
400/400 [==============================] - 33s 84ms/step - loss: 0.2305 - accuracy: 0.9092 - val_loss: 0.2845 - val_accuracy: 0.8784
Epoch 3/3
400/400 [==============================] - 33s 82ms/step - loss: 0.1849 - accuracy: 0.9293 - val_loss: 0.3060 - val_accuracy: 0.8853


INFO:tensorflow:Assets written to: sentiment-score-model\assets


INFO:tensorflow:Assets written to: sentiment-score-model\assets


In [ ]:
movie = input("Enter your movie name: ")
movie_hashtag='#'+movie.replace('/n','').replace(' ','')
print("Looking For Tweets: ", movie_hashtag)

Enter your movie name: Spider Man No Way Home
Looking For Tweets:  #SpiderManNoWayHome


In [ ]:
#Getting Twitter Feed:
import tweepy
import config

client=tweepy.Client(bearer_token=config.BEARER_TOKEN)
query='review '+movie_hashtag+' -is:retweet lang:en -has:media'
response=client.search_recent_tweets(query=query, max_results=10)
#print(response)


Response(data=[<Tweet id=1515075755033604105 text='@ScottFeinberg We dropped 2 pods on #SpiderManNoWayHome \n\nInitial review:\n\nApple https://t.co/sr0sfdXCPt\nSpotify https://t.co/uETgzVVYdr\n\nFollow up discussing the Multiverse:\n\nApple https://t.co/bATGfAhRgu\nSpotify https://t.co/tTcucCiIze'>, <Tweet id=1515024930286764032 text="My ★★★★½ review of Spider-Man: No Way Home on Letterboxd\n\nAbsolutely Tom Holland's best run as #SpiderMan\n\n https://t.co/6OBiJQvA3b\n\n#SpiderManNoWayHome">, <Tweet id=1514617111192096768 text='The Dan Richardson Show Episode 175: Spider-Man: No Way Home Spoiler Review\nhttps://t.co/VjTPPUX11C\n\nIn which, I finally do a spoiler review for #SpiderManNoWayHome In honor of the Blu Ray release!!!!'>, <Tweet id=1514585181235163136 text='My review of the superhero sequel… #SpiderManNoWayHome #TomHolland https://t.co/nR2I98KCqz'>, <Tweet id=1514573477201727489 text='Spider-Man: No Way Home has arrived on the 4K UHD format and Cereal At Midnight has your in

In [ ]:
lst_movie_reviews_from_twtr=[]
for tweet in response.data:
    lst_movie_reviews_from_twtr.append(tweet.text)

In [ ]:
len(lst_movie_reviews_from_twtr)

10

In [ ]:
from tensorflow import keras
reconstructed_model = keras.models.load_model("sentiment-score-model")


#ex_text_str = "Another cash cow from Marvel. Never mind with a quality script. Let's just pump this crap out as fast as possible to cash in on the fame of Marvel. It's sick to know that Stan Lee sold out. This movie deserves negative stars. It sucks that bad."

data_test = {'review':lst_movie_reviews_from_twtr}
df_test = pd.DataFrame(data=data_test)
df_test.head()
df_test["review"]=df_test.review.apply(lambda x: clean_text(x))

list_sentences_test = df_test["review"]
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
X_te = pad_sequences(list_tokenized_test, maxlen=maxlen)
prediction = reconstructed_model.predict(X_te)
print("prediction:",prediction)

prediction: [[0.19055247]
 [0.28203535]
 [0.67855877]
 [0.53518677]
 [0.8954531 ]
 [0.13322309]
 [0.04018784]
 [0.8442749 ]
 [0.02998617]
 [0.822474  ]]


In [ ]:
df_test['Sentiment Score']=prediction
df_test

,review,Sentiment Score
0,@ScottFeinberg We drop 2 pod #SpiderManNoWayHo...,0.190552
1,My ★★★★½ review Spider-Man: No Way Home Letter...,0.282035
2,The Dan Richardson Show Episode 175: Spider-Ma...,0.678559
3,My review superhero sequel… #SpiderManNoWayHom...,0.535187
4,Spider-Man: No Way Home ha arrive 4K UHD forma...,0.895453
5,New #HotToys unbox &amp; review Youtube channe...,0.133223
6,Is MORBIUS WORST superhero movie EVER?!? -- HA...,0.040188
7,Hot Toys Review | Spider-Man Black &amp; Gold ...,0.844275
8,Is safe review Spider-Man No Way Home YouTube ...,0.029986
9,"#SpiderManNoWayHome available 4K, Blu-Ray, DVD...",0.822474


In [ ]:
p_mean=prediction.mean()
if p_mean>=0.6:
    print('Must Watch','Average Review Score:',p_mean*100,'%')
if p_mean>0.4 and p_mean<0.6:
    print('Maybe Watch','Average Review Score:',p_mean*100,'%')
if p_mean<=0.4:
    print('Not Worth It','Average Review Score:',p_mean*100,'%')

Maybe Watch Average Review Score: 44.519323110580444 %
